In [1]:
import numpy as np

#add ../ to the path
import sys
sys.path.append('../')

from sudoku_examples import easy, hard
easy = np.array(easy)
hard = np.array(hard)

In [2]:
print(easy)

[[5 3 0 0 7 0 0 0 0]
 [6 0 0 1 9 5 0 0 0]
 [0 9 8 0 0 0 0 6 0]
 [8 0 0 0 6 0 0 0 3]
 [4 0 0 8 0 3 0 0 1]
 [7 0 0 0 2 0 0 0 6]
 [0 6 0 0 0 0 2 8 0]
 [0 0 0 4 1 9 0 0 5]
 [0 0 0 0 8 0 0 7 9]]


In [3]:
print(hard)

[[0 0 6 1 0 0 0 0 8]
 [0 8 0 0 9 0 0 3 0]
 [2 0 0 0 0 5 4 0 0]
 [4 0 0 0 0 1 8 0 0]
 [0 3 0 0 7 0 0 4 0]
 [0 0 7 9 0 0 0 0 3]
 [0 0 8 4 0 0 0 0 6]
 [0 2 0 0 5 0 0 8 0]
 [1 0 0 0 0 2 5 0 0]]


In [4]:
medium = np.array([
    [0, 6, 7, 0, 8, 2, 9, 5, 0],
    [1, 0, 0, 6, 0, 5, 7, 0, 8],
    [0, 0, 8, 7, 1, 0, 0, 6, 0],
    [0, 5, 0, 0, 7, 0, 0, 9, 0],
    [0, 7, 9, 0, 5, 0, 0, 0, 6],
    [2, 0, 0, 8, 0, 6, 0, 7, 0],
    [0, 0, 2, 0, 6, 0, 5, 8, 0],
    [9, 0, 6, 5, 0, 8, 0, 3, 7],
    [0, 0, 5, 0, 0, 7, 6, 1, 0]
])

In [5]:
# create new sudoku with predetermined values filled in
def connected_values(puzzle, i, j):
    connected_rows = puzzle[i, :]
    connected_columns = puzzle[:, j].flatten()
    connected_square = puzzle[i//3*3:i//3*3+3, j//3*3:j//3*3+3].flatten()
    connected_values = np.unique(np.concatenate((connected_rows, connected_columns, connected_square)))
    connected_values = connected_values[connected_values != 0]
    return connected_values

def fill_puzzle_first_order(puzzle):
    new_puzzle = puzzle.copy()
    for i, j in np.argwhere(puzzle == 0):
        connected_values_list = connected_values(puzzle, i, j)
        possible_values = np.setdiff1d(np.arange(1, 10), connected_values_list)
        if len(possible_values) == 1:
            new_puzzle[i, j] = possible_values[0]
    return new_puzzle

def prefill_puzzle(puzzle):
    current_puzzle = puzzle.copy()
    new_puzzle = fill_puzzle_first_order(current_puzzle)

    while not np.array_equal(current_puzzle, new_puzzle):
        current_puzzle = new_puzzle.copy()
        new_puzzle = fill_puzzle_first_order(current_puzzle)
    return new_puzzle


In [6]:
hard_pre_filled = prefill_puzzle(hard)
print(hard_pre_filled)

[[0 0 6 1 0 0 0 0 8]
 [0 8 0 0 9 0 0 3 0]
 [2 0 0 0 0 5 4 0 0]
 [4 0 0 0 0 1 8 0 0]
 [0 3 0 0 7 0 0 4 0]
 [0 0 7 9 0 0 0 0 3]
 [0 0 8 4 0 0 0 0 6]
 [0 2 0 0 5 0 0 8 0]
 [1 0 0 0 0 2 5 0 0]]


In [7]:
print(f"number of added cells: {len(np.argwhere(hard_pre_filled != hard))}")   

number of added cells: 0


In [8]:
easy_pre_filled = prefill_puzzle(easy)
print(easy_pre_filled)

[[5 3 4 6 7 8 9 1 2]
 [6 7 2 1 9 5 3 4 8]
 [1 9 8 3 4 2 5 6 7]
 [8 5 9 7 6 1 4 2 3]
 [4 2 6 8 5 3 7 9 1]
 [7 1 3 9 2 4 8 5 6]
 [9 6 1 5 3 7 2 8 4]
 [2 8 7 4 1 9 6 3 5]
 [3 4 5 2 8 6 1 7 9]]


In [9]:
print(f"number of added cells: {len(np.argwhere(easy_pre_filled != easy))}")

number of added cells: 51


In [10]:
import time as time

def return_possible_values_matrix_naked_pair(puzzle):
    possible_values_matrix = np.zeros((9, 9, 9), dtype=bool)
    for i, j in np.argwhere(puzzle == 0):
        connected_values_list = connected_values(puzzle, i, j)
        possible_values = np.setdiff1d(np.arange(1, 10), connected_values_list)
        possible_values_matrix[i, j] = np.isin(np.arange(1, 10), possible_values)

    for i, j in np.argwhere(puzzle == 0):
        for k, l in np.argwhere(puzzle == 0):
            if (i, j) != (k, l):
                intersect = np.logical_and(possible_values_matrix[i, j], possible_values_matrix[k, l])
                if np.sum(intersect) == 2:
                    possible_values_matrix[i, j] = intersect
                    possible_values_matrix[k, l] = intersect
    return possible_values_matrix

def prefill_puzzle_naked_pair(puzzle):
    current_puzzle = puzzle.copy()

    possible_values_matrix = return_possible_values_matrix_naked_pair(current_puzzle)
    new_puzzle = puzzle.copy()
    indices_to_fill = np.argwhere(np.sum(possible_values_matrix, axis=2) == 1)
    for i, j in indices_to_fill:
        new_puzzle[i, j] = np.argmax(possible_values_matrix[i, j]) + 1

    while not np.array_equal(current_puzzle, new_puzzle):
        current_puzzle = new_puzzle.copy()

        possible_values_matrix = return_possible_values_matrix_naked_pair(current_puzzle)
        new_puzzle = current_puzzle.copy()
        indices_to_fill = np.argwhere(np.sum(possible_values_matrix, axis=2) == 1)
        for i, j in indices_to_fill:
            new_puzzle[i, j] = np.argmax(possible_values_matrix[i, j]) + 1
    return new_puzzle